<a href="https://colab.research.google.com/github/jkorgu/C4145CW/blob/main/CE4145cw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Section One - Dataset__

<br>

## __Dataset Information__

The dataset is called "rotten_tomatoes" and it is provided by "cornell-movie-review-data" which was published on the website "huggingface".

The dataset itself is a collection of 10,662 rotten tomatoes movie reviews, split into 2 categories of positive and negative, labeled 1 and 0 respectively.

Furthermore, the dataset splits into "train", "validation" and "test" files which saves me from creating the splits myself, however I still put them into seperate dataframes.

<br>

## __Task to be performed__

The task performed in this coursework is text classification with the main goal of being able to predict the appropriate label as output through the content of the text within the review as input.

__113 words total__

In [ ]:
#vscode previous machine learning project layout:
#1.1dataset
#1.2 task
#1.3 data exploration
#1.3.1 basic data exploration (shape, head)
#1.3.2 names of features/classes (renaming columns, wont do this here)
#1.3.3 class distribution (check class distribution (label here per split)) (more is done but that is irrelevant here)
#1.3.4 check missing values
#1.3.5 outlier values (wont be needed here)
#1.3.6 correlations (wont be needed here)
#1.4 data pre-process
#1.4.1 data erasure (Wont be needed here)
#1.4.2 seperatinf datasets (wont be needed here)
#1.4.3 data split (done for me)
#2 modelling/classification
#2.1 result interpretation (Comparing results w paper and takling about current accuracy, not needed here)
#3 solution improvement
#3.1 hyperparameter tuning - probably neeeded here
#3.2 feature importance - not needed here
#3.3 model comparison - yeah but set it out differentlys
#3.3.1 result interepretation - yeah
#3.3.2 feature importance - no

#possible layouts
#1 dataset
#1.1 dataset exploration
#1.1.1 basic data exploration (shape, head, names of features/classes, missing values)
#1.1.2 class distribution (label per split, 3 bar graphs to confirm what was said earlier about dataset)
#1.2 preprocessing
#2 representation learning
#3 algorithms
#3.1 tuning?
#3.2 model comparison - bar graph?
#4 evaluation
#4.1 result interpretation?

## 1.1 Dataset Exploration

In this section we will be exploring the dataset to confirm its shape, size and other factors to ensure it matches the description given online.

#### 1.1.1 Basic Data Exploration

Here we will look at these parts of the dataset:
- Shape
- Samples of the data (df.head())
- Check for missing values

In [1]:
import pandas as pd
import numpy as np

#import dataset from huggingface, splits are already decided on the website, as seen below with our seperate dataframes
splits = {'train': 'train.parquet', 'validation': 'validation.parquet', 'test': 'test.parquet'}
dfTrain = pd.read_parquet("hf://datasets/cornell-movie-review-data/rotten_tomatoes/" + splits["train"])
dfValidation = pd.read_parquet("hf://datasets/cornell-movie-review-data/rotten_tomatoes/" + splits["validation"])
dfTest = pd.read_parquet("hf://datasets/cornell-movie-review-data/rotten_tomatoes/" + splits["test"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [24]:
#Shape of the training dataset
dfTrain.shape

(8530, 2)

In [25]:
#Shape of the validation dataset
dfValidation.shape

(1066, 2)

In [27]:
#Shape of the text dataset
#to confirm 8530+1066+1066=10,662 rows total
dfTest.shape

(1066, 2)

In [2]:
#Show the training data, both positive and negative entries (confirming text and label columns too)
dfTrain.head(-5)

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
8520,"the star who helped give a spark to "" chasing ...",0
8521,a summary of the plot doesn't quite do justice...,0
8522,every conceivable mistake a director could mak...,0
8523,"snoots will no doubt rally to its cause , trot...",0


### 1.2 Preprocessing

Here we will import the natural language toolkit (NLTK) to tokenize and do some simple cleaning on the data

In [4]:
#Creating our data splits for later training, testing and validation of pipelines

#train
xTrain = dfTrain['text']
yTrain = dfTrain['label']

#validation
xValidation = dfValidation['text']
yValidation = dfValidation['label']

#test
xTest = dfTest['text']
yTest = dfTest['label']

In [22]:
#coursework code for inspo+testing
#rework this to be original, and to make it work for your data (no punctuation removal?)
#also if no countvec, need to rework bottom part possibly
#if code kept the same as example grade gets capped to C

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

def prep(X):
  prepText = []
  for x in X:
    tokenText = word_tokenize(x) #tokenize doc
    normdText = [token.lower() for token in tokenText] #list compression to apply some simple cleaning (lower case casting and punctuation removal) to tokenized terms

    #important
    swrText = [token for token in tokenText if token not in stopwords.words('english')] #list compression to remove stopwords from our list

    stemmer = SnowballStemmer("english") #specify english stemming rules
    prepText += [[stemmer.stem(word) for word in swrText]] #list compression for applying stemmer

  prepSentences = [" ".join(sentence) for sentence in prepText] #join sentences back together to ensure compatibility with countvec
  return prepSentences

#prepped x
prepXTrain = prep(xTrain)
prepXValidation = prep(xValidation)
prepXTest = prep(xTest)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
print(xTrain[0])

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .


In [21]:
print(prepXTrain[0])
#rock destin 21st centuri 's new `` conan `` 's go make splash even greater arnold schwarzenegg , jean-claud van damm steven segal . - all enabled, no normd same output

rock destin 21st centuri 's new `` conan `` 's go make splash even greater arnold schwarzenegg , jean-claud van damm steven segal .


#Section Two - Representation Learning

#Section Three - Algorithms

#Section Four - Evaluation